In [ ]:
%conda install transformers accelerate

In [ ]:
print(torch.__version__)

In [1]:
import os
import sagemaker
import boto3
import tarfile

In [ ]:
sess = sagemaker.Session()
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

In [6]:
# Create model directory to save to
model_dir = "./models/"
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

In [7]:
student_id = "gpt2"
teacher_id = "gpt2-medium"
dataset_id = "glue"
dataset_config = "sst2"

In [ ]:
trainer.train()
trainer.save_model(model_dir)

In [28]:
s3 = boto3.client('s3')

In [ ]:
s3_model_path = 'results'
tar_gz_file = "model.tar.gz"

In [ ]:
with tarfile.open(tar_gz_file, "w:gz") as tar:
    tar.add(model_dir, arcname=os.path.basename(model_dir))

print(f"Compressed and archived {model_dir} to {tar_gz_file}")

In [ ]:
model_file = os.path.join(".", tar_gz_file)
s3_file = f"{sess.default_bucket()}/{s3_model_path}/{tar_gz_file}"

try:
    print(f'Uploading {model_file} to {s3_file}...')
    s3.upload_file(model_file, sess.default_bucket(), f"{s3_model_path}/{tar_gz_file}")
    print(f'Uploaded {model_file} to {s3_file}')
except Exception as e:
    print(f"Error occurred while uploading file {model_file}, {e}")